# Area statistics for protected areas in mainland Norway

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb)

**Author**: Willeke A'Campo

**Description:** This notebook calculates area statistics for protected areas in mainland Norway. Area statistics are calculated for the datasetes **Verneområder** og **Foreslatte verneområder** from the Norwegian Environment Agency. 

The area statistics can be divided in three groups:

1. Area variables for the protected areas:
    - Area (terrestrial and marine)
    - Perimeter (terrestrial and marine)
    - Land area (terrestrial)
    - Perimeter land area (terrestrial)

2. Overlay statistics for the protected area and the following datasets:
    - AR50 - Bonitet
        - Area proportion of land cover quality classes 
    - Bioklimatiske soner
        - Area proportion of bioclimatic zone classes
    - Infrastrukturindeks: 
        - Mean value of infrastructure index per protected area 
        - Area proportion of infrastructure index classes
    - Høydelag
        - Area proportion of elevation classes 

3. Spatial indices for the protected areas:
    - Density of protected area per 10x10km (SSB grid)
    - Shape Index for each protected area (land + marine)
    - Shape index for each protected area (land)






In [1]:
import os 
from osgeo import gdal
from pathlib import Path
from itertools import islice

from shapely.geometry import box
import pandas as pd
import geopandas as gpd

# mapping libraries
# https://leafmap.org/faq/#how-to-use-a-specific-plotting-backend
import leafmap.foliumap as leafmap

In [2]:
gis_server = True
project_path= Path.cwd().parents[0]
shell_path= os.path.join(project_path, "src", "shell")
python_path= os.path.join(project_path, "src", "python")

if gis_server:
    data_path = "/home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data"
else:
    data_path = os.path.join(project_path, "data")

print(f"Gdal Version: {gdal.__version__} ")
print(f"Project Path: {project_path}")
print(f"Path to shell scripts: {shell_path}")
print(f"Path to python scripts: {python_path}")
print(f"Path to data: {data_path}")

Gdal Version: 3.8.3 
Project Path: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer
Path to shell scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/shell
Path to python scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/python
Path to data: /home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data


In [3]:
# import local scripts
import sys 
sys.path.append(python_path)

from ogr_utils import import_gpkg, print_layer_schema
from lookup import create_lookup_dict, lookup_value
from duckdb_utils import load_gpkg_layers, ar50_area_class, print_duckdb_info

## Import Data

Get the data and import into geopackages `vern_25833.gpkg` and `bevaring_25833.gpkg` using ogr2ogr.

```python
# change path to data folder
os.chdir(os.path.join(data_path, "raw"))
print(f"Download data to: {os.getcwd()}")
```

```bash
# ArcGIS REST: Naturvern (02.2024)
ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln naturvernomrade -nlt MULTIPOLYGON vern_25833.gpkg "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/0/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

# ArcGIS REST: Foreslåtte verneområder (02.2024)
ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln foreslatt_vern -nlt MULTIPOLYGON vern_25833.gpkg "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/4/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

# AR50 - 2022 
# Stored on NINA's server /GeoSpatialData

# Bioklima
# Received from V. Bakkestuen (NINA)

# SSB grid
curl -o Ruter_10KM_norge.zip "https://www.ssb.no/natur-og-miljo/_attachment/375082?_ts=1685c0e69b8"
unzip Ruter_10KM_norge.zip

# Fylkerområder
# Stored on NINA's server /GeoSpatialData

# Infrastruktur
# Received from V. Bakkestuen (NINA)
```

Downloaded datasets are stored in the `/data` folder:

**vern_25833.gpkg** contains the following layers:
| Layer | Dataset Name | Description | Year | Source |
|-------| ------------ | ----------- | ---- | ------ |
| naturvernomrade | Naturvernområder| Nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/0) |
| foreslatt_vern | Foreslåtte naturvernområder | Planned nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/1)| 

**bevaring_25833.gpkg** contains the following layers:
|Layer| Dataset Name | Description | Year | Source |
|-----| ------------ | ----------- | ---- | ------ |
| ar50_2022 | AR50 | Land cover classes | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard) |
| bioklima_soner_2017 | Bioklimatiske soner | Bioclimatic zones | 2017 | [Artsdatabanken](https://data.artsdatabanken.no/Natur_i_Norge/Natursystem/Beskrivelsessystem/Regional_naturvariasjon/Bioklimatisk_sone) |

**other files**

| Filename | Dataset Name | Description | Year | Source |
|----------| ------------ | ----------- | ---- | ------ |
| ssb_grid_5km.geojson | SSB rutenett (5x5 km)| Grid for Norway | 2024 | [SSB](https://kart.ssb.no/) |
| fylker_2024.geojson | Fylker, 2024 | Provincial Boundaries | 2024 | [SSB](https://www.ssb.no/en/kart/griddata) |
| Infra25m.tif | Infrastrukturindeks | Infrastructure index (25m)| 2024 | Internal NINA datasett |

In [4]:
run_cell = False

if run_cell: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)

    input_file = "vern_25833.gpkg"
    path = os.path.join(data_path, "interim")

    ! gdalinfo --version
    ! chmod +x gdal_gpkg-info.sh
    ! ./gdal_gpkg-info.sh {path} {input_file}

## Data Preparation

### Case Study | Fosen, Trøndelag

In this notebook we use Fosen as an example site to follow along the analysis steps on the map. Note that the analysis is performed for all protected areas in mainland Norway. 

https://ipyleaflet.readthedocs.io/en/latest/installation/index.html#using-pip 


### Prepare bounding box

In [5]:
# bounding box Dovrefjell
bounding_box = "fosen"  # "dovrefjell" or "trondheim"

if bounding_box == "dovrefjell":
    xmin, ymin, xmax, ymax = 160000.00, 6900000.00, 260000.00, 6950000.00
if bounding_box == "fosen":
    xmin, ymin, xmax, ymax = 180000.00, 7010000.00, 290000.00, 7150000.00
if bounding_box == "trondheim":
    xmin, ymin, xmax, ymax = 260520.12, 7032142.5, 278587.56, 7045245.27

# Create a bounding box
boxBB =  box(xmin, ymin, xmax, ymax)
crs = 'EPSG:25833'

gdf_BB = gpd.GeoDataFrame(geometry=[boxBB])
gdf_BB['name'] = 'Dovre_BB'
gdf_BB.crs = crs
bounds = gdf_BB.bounds.to_numpy().tolist()[0]

In [6]:
run_cell = False

if run_cell: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)
    for file in ["vern", "bevaring", "admin"]:
        input_file = os.path.join(data_path, "interim", f"{file}_25833.gpkg")
        output_file = os.path.join(data_path, "tmp", f"{file}_25833_bbox.gpkg")
        
        #for layer_name in layer_names:
        ! chmod +x gdal_copy-file-bbox.sh
        ! ./gdal_copy-file-bbox.sh {input_file} {output_file} {xmin} {ymin} {xmax} {ymax}

### Display Data on the Map

In [7]:
# Display data on map using leafmap
init_location = [62.223207, 9.550195]  # Hjerkinn
zoom_start = 13  
min_zoom = 8  # keeps user from zooming out too far
basemap = leafmap.Map(
    location=init_location, 
    zoom=zoom_start, 
    min_zoom=min_zoom, 
    max_bounds=True
    )

# set background
basemap.add_basemap("SATELLITE", opacity=0.7)

# add WMS verneområder
wms_url ="https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer"
wms_layer = "naturvern_klasser_omrade"
wms_name = "naturvern_klasser_omrade"
basemap.add_wms_layer(
    url=wms_url, 
    layers=wms_layer, 
    name=wms_name, 
    wms_format="image/png",
    )
# add bounding box - Hjerkinn
basemap.add_gdf(gdf_BB, layer_name="Dovre_BB", fill_color="blue", fill_opacity=0.2, weight=2)

basemap

### AR50 - Bonitet

Translate the Bonitet classes using a lookup table.
 
&emsp; $Bonitet = (artype, artreslag, arskogbon, arjordbr, ardyrkning, arveget)$ 

In [8]:
run_cell = False

if run_cell: 
    # create a temporary test gpkg
    tmp_gpkg = os.path.join(data_path, "tmp", "tmp.gpkg")
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    print(tmp_gpkg)

    sql_query = f'SELECT * FROM "ar50_flate" LIMIT 5'
    path = os.path.join(data_path, "tmp")
    ! ogr2ogr -f "GPKG" -nln "ar50_flate" -sql "$sql_query" {tmp_gpkg} {in_gpkg}

In [9]:
# if Test is True, import test gpkg otherwise import the original gpkg
study_area = True

# ar50 layer
layer_name, new_field_name = "ar50_flate", "ar50_bonitet"

if study_area:
    # import gpkg into ogr object 
    # add field name if it does not exist
    tmp_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
    ds, lyr = import_gpkg(tmp_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)

else:
    # import gpkg into ogr object 
    # add field name if it does not exist
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    ds, lyr = import_gpkg(in_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)

# Convert the first 5 features of the layer to a DataFrame
# use islice to limit the number of features to 5, to reduce computation time
df_AR50 = pd.DataFrame(feature.items() for feature in islice(lyr, 5))

# Print the DataFrame
df_AR50

,OBJECTID,objtype,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,arkartstd,målemetode,informasjon,oppdateringsdato,originaldatavert,navnerom,kopidato,områdeid,lokalid,fylkeid,komid,ar50_bonitet
0,1281054,arealressursflate,81,98,98,98,98,98,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,3432,1281054,34,3432,16
1,1281096,arealressursflate,50,98,39,51,98,99,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,3432,1281096,34,3432,13
2,1281060,arealressursflate,50,98,39,51,98,99,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,3432,1281060,34,3432,13
3,1281014,arealressursflate,50,98,39,51,98,99,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,3432,1281014,34,3432,13
4,1281068,arealressursflate,81,98,98,98,98,98,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,3432,1281068,34,3432,16


In [10]:
# lookup table 
lookup_csv = os.path.join(data_path, "AR50_bonitet_lookup.csv")
lookup_df = pd.read_csv(lookup_csv)

# rename cols to correspond with AR50 lyr
lookup_df.rename(
    columns={
        "ARTYPE kode": "artype",
        "ARTRESLAG kode": "artreslag",
        "ARSKOGBON kode": "arskogbon",
        "ARJORDBR kode": "arjordbr",
        "ARDYRKING kode": "ardyrking",
        "ARVEGET kode": "arveget",
        "Bonitet kode": "ar50_bonitet",
    },
    inplace=True,
)

# reorder cols to correspond with AR50 lyr
lookup_df = lookup_df[
    [
        "artype",
        "arskogbon",
        "artreslag",
        "arveget",
        "arjordbr",
        "ardyrking",
        "Beskrivelse",
        "ar50_bonitet",
    ]
]

display(lookup_df.head(11))

,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,Beskrivelse,ar50_bonitet
0,10,98,98,98,98,98,Bebygd og samferdsel,14
1,20,98,98,98,24,98,Fulldyrka og Overflatedyrka jord,1
2,20,98,98,98,25,82,"Innmarksbeite, dyrkbar jord",2
3,20,98,98,98,25,81,Innmarksbeite,2
4,20,98,98,98,99,99,"Jordbruksareal, uspesifisert",1
5,30,11,31,98,98,82,"Barskog, impediment, dyrkbar jord",6
6,30,11,31,98,98,81,"Barskog, impediment",6
7,30,12,31,98,98,82,"Barskog, lav skogbonitet, dyrkbar jord",5
8,30,12,31,98,98,81,"Barskog, lav skogbonitet",5
9,30,13,31,98,98,82,"Barskog, middels skogbonitet, dyrkbar jord",4


In [11]:
# create lookup dict
# keys must be in same order as gpkg lyr fields
keys = (
    "artype",
    "arskogbon",
    "artreslag",
    "arveget",
    "arjordbr",
    "ardyrking",
    )

value= "ar50_bonitet"

lookup_dict = create_lookup_dict(
    lookup_df,
    keys=keys,
    value=value
)

# print first two entries of dict
print({k: lookup_dict[k] for k in list(lookup_dict)[:11]})

{(10, 98, 98, 98, 98, 98): 14, (20, 98, 98, 98, 24, 98): 1, (20, 98, 98, 98, 25, 82): 2, (20, 98, 98, 98, 25, 81): 2, (20, 98, 98, 98, 99, 99): 1, (30, 11, 31, 98, 98, 82): 6, (30, 11, 31, 98, 98, 81): 6, (30, 12, 31, 98, 98, 82): 5, (30, 12, 31, 98, 98, 81): 5, (30, 13, 31, 98, 98, 82): 4, (30, 13, 31, 98, 98, 81): 4}


In [12]:
run_cell = False

if run_cell:
    # loop through the features and reclassify the attribute value "ar50_bonitet"
    features_to_update = []

    for feature in lyr:
        # get the attribute values
        artype = feature.GetField("artype")
        arskogbon = feature.GetField("arskogbon")
        artreslag = feature.GetField("artreslag")
        arveget = feature.GetField("arveget")
        arjordbr = feature.GetField("arjordbr")
        ardyrking = feature.GetField("ardyrking")

        key = (int(artype), int(arskogbon), int(artreslag), int(arveget), int(arjordbr), int(ardyrking))
        if key in lookup_dict:
            new_value = lookup_dict[key]
            feature.SetField("ar50_bonitet", new_value)
            features_to_update.append(feature)

    print(f"Number of features to update: {len(features_to_update)}")
    # Batch update features
    for feature in features_to_update:
        lyr.SetFeature(feature)

    print("Finished updating Bonitet.")

    df_AR50_bon = pd.DataFrame(feature.items() for feature in islice(lyr, 5))
    display(df_AR50_bon.head())

# close OGR object
ds = None

In [13]:
run_cell = False

if run_cell:
    # Define the SQL queries
    sql_marine = f"SELECT * FROM ar50_flate WHERE ar50_bonitet = 17"
    sql_terrestisk = f"SELECT * FROM ar50_flate WHERE ar50_bonitet != 17"

    # if Test is True, import test gpkg otherwise import the original gpkg
    study_area = True


    if study_area:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        out_gpkg = os.path.join(data_path, "tmp", "interim_25833_bbox.gpkg")
    else:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        out_gpkg = os.path.join(data_path, "interim", "interim_25833.gpkg")

    print(out_gpkg)

    # Create the ar50_marine layer
    ! ogr2ogr -f "GPKG" -nln "ar50_marine" -sql "$sql_marine" -append {out_gpkg} {in_gpkg}
    # Create the ar50_terrestisk layer
    ! ogr2ogr -f "GPKG" -nln "ar50_terrestisk" -sql "$sql_terrestisk" -append {out_gpkg} {in_gpkg}

In [14]:
# AR50 into duckdb
# if Test is True, import test gpkg otherwise import the original gpkg
study_area = True

if study_area:
    in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
    db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
    load_gpkg_layers(db_path, in_gpkg, "ar50_flate")
    load_gpkg_layers(db_path, in_gpkg, "ar50_flate_msk")
else:
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
    load_gpkg_layers(db_path, in_gpkg, "ar50_flate")

Table ar50_flate already exists in DuckDB database. Skipping.
Table ar50_flate_msk already exists in DuckDB database. Skipping.


### Protected Areas

Divide the protected areas into terrestrial and marine areas

Bonitet = 17 (Hav) are considered as marine areas, all other areas are considered as terrestrial areas.
ARealtype = 82 (Hav)

In [15]:
# import protected areas into DuckDB

# if Test is True, import test gpkg otherwise import the original gpkg
study_area = True

if study_area:
    in_gpkg = os.path.join(data_path, "tmp", "vern_25833_bbox.gpkg")
    db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
    load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
    load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")
else:
    in_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
    db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
    load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
    load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")

Table naturvernomrade already exists in DuckDB database. Skipping.
Table foreslatt_vern already exists in DuckDB database. Skipping.


### Infrastructure index

- Calculate and display the distribution of the infrastructure index for the whole of norway 
- Calculate and display the distribution of the infrastructure index per region
- Calculate and display the distribution of the infrastructure index per protected area
- Define the infrastructure index classes


### Topographic height

Classify the topographic height into 4 classes:
- 0-300m
- 301-600m
- 601-900m
- over 900m

## Methods
### 1. Geometry variables for protected areas

| Protected area | Total area (km2) | Perimeter (km) | Land area (km2) | Perimeter land area (km) |
|----------------|------------|----------------|-----------------|-------------------------|
| *NaturvernId* | *areal_km2* | *omkrets_km* | *landareal_km2* | *landomkrets_km*|
| area A         |    *        |                |                 |                         |

In [16]:
# calc. landareal = area that overlaps with ar50_terrestrial
# calc. marineareal = area that overlaps with ar50_marine



***
$\mathbf{\text{Shape Index}}$<br>
***
The shape index is a measure of how compact the shape is compared to a circle with the same area. The shape index is calculated for the entire protected area and the land area. The shape index is a value between 0 and 1, where 0 is a perfect circle and 1 is a long and narrow shape.

- $P$ = perimeter
- $r$ = radius
- $A$ = area


**Shape Index:**&emsp;      $SI = \frac{P}{2\pi r}$

**Radius:**&emsp; $r = \sqrt{\frac{A}{\pi}}$

In [17]:
# calc formindex

### 2. Area overlay statistics for the protected area and the following datasets:





**AR50 - Bonitet**

| Protected area | Fulldyrka og overflatedyrka jord (%) | Innmarksbeite (%) | Skog, høg og særs høg bonitet (%) | ... |
|----------------|--------------------------------------|-------------------|-----------------------------------|-----|
| *NaturvernId*  |*AR50_bon_1*|*AR50_bon_2*|*AR50_bon_3*|...|
| area A         |                                      |                   |                                   |     |


In [18]:
import duckdb
print_duckdb_info(db_path)

# print columnnames of duckdb table "forelslat_vern"
with duckdb.connect(db_path) as con:
    df = con.execute("SELECT * FROM ar50_flate_msk").fetchdf()
    print(df.columns)

Tables: [('ar50_flate',), ('ar50_flate_msk',), ('foreslatt_vern',), ('naturvernomrade',)]
Index(['OBJECTID', 'objtype', 'artype', 'arskogbon', 'artreslag', 'arveget',
       'arjordbr', 'ardyrking', 'arkartstd', 'målemetode', 'informasjon',
       'oppdateringsdato', 'originaldatavert', 'navnerom', 'kopidato',
       'områdeid', 'lokalid', 'fylkeid', 'komid', 'ar50_bonitet', 'geometry'],
      dtype='object')


In [19]:
tbl_study_area = "foreslatt_vern"
id = "identifikasjon_lokalId"
tbl_ar50 = "ar50_flate_msk"
ar50_field = "ar50_bonitet"
area_class = 17
new_field = "ar50_bon17"


ar50_area_class(db_path, tbl_study_area, id, tbl_ar50, ar50_field, area_class, new_field)

An error occurred: Binder Error: No function matches the given name and argument types 'ST_Intersects(BLOB, BLOB)'. You might need to add explicit type casts.
	Candidate functions:
	ST_Intersects(BOX_2D, BOX_2D) -> BOOLEAN
	ST_Intersects(GEOMETRY, GEOMETRY) -> BOOLEAN

LINE 11:                     ST_Intersects( study_area.geometry, ar50_overlap.geometry )
                GROUP BY study_area.identifikasjon_lokalId
            ...
                             ^


In [ ]:
# print first 5 rows of the table "foreslatt_vern"
with duckdb.connect(db_path) as con:
    df = con.execute("SELECT * FROM foreslatt_vern").fetchdf()
    display(df.head())

**Bioklimatiske soner**

| Protected area | Sone 1 (%) | Sone 2 (%) | Sone 3 (%) | ... |
|----------------|------------|------------|------------|-----|
| *NaturvernId*  |*bioklima_1*|*bioklima_2*|*bioklima_3*|...  |
| area A         |            |            |            |     |

**Infrastrukturindeks**

| Protected area | Mean value | Lav (%) | Middels (%) | Høy (%) | ... |
|----------------|------------|---------|-------------|---------|-----|
| area A         |            |         |             |         |     |


**Høydelag**

| Protected area | 0-300 m (%) | 301-600 m (%) | 601-900 m (%) | over 900 m (%) |
|----------------|-------------|---------------|---------------|----------------|
| area A         |             |               |               |                |


### 3. Regional Statistics


Administrative regions:
- Municipalities
- Counties
- Regions 
    - **Nord**: Finnmark, Troms, Nordland
    - **Midt**: Trøndelag, Møre og Romsdal
    - **Sør**: Agder, Vestfold, Telemark
    - **Øst**: Østfold, Akershus, Oslo, Innlandet, Buskerud
    - **Vest**: Vestland, Rogaland


| Protected area | Municipality | County | Region |
|----------------|---------|-------|--------|
| *NaturvernId*  |*kommune*|*fylke*|*region*|
| area A         |         |       |        |

*Regional datasett*:

| Region | Area | Protected Area | Land Protected Area | Marine Protected Area |
|--------|------|----------------|---------------------|-----------------------|
| *Region* | *areal* | *a_vern* | *a_landvern* | *a_marinvern* |
| Nord ||||||


###  4. Grid Statistics (10x10 km2)

**Rutenettstatistikk:**

| SSB grid cell | area | Density (land + marine) | Density (land) | 
| --------------|------|-------------------------|----------------|
| *grid_ID* | *grid_areal* | *tetthet_tot_vern* | *tetthet_landvern* | 
| cell 1| 100 km2 | ||





***
$\mathbf{\text{Density}}$<br>
***
The density of protected area per 10x10km (SSB grid) is calculated by dividing the area of the protected area by 100 km^2.

**Density:**&emsp;      $Density = \frac{A}{100 km^2}$



In [20]:
# code for density calculation here
# check land cover fraction notebook
